In [ ]:
import polars as pl
import numpy as np
import polars_ds as pld

In [ ]:
df = pl.DataFrame({
    "a":["AAAAA", "AAATT", "ATTTT", "AAAAA", "AAAAA"],
    "b":["AAAAT", "AAAAA", "ATATA", "AAAAA", "TTTTT"]
})

In [ ]:
df.select(
    pl.col("a").str2.similar_words(pl.col("b"), k = 2, metric = "lv")
)

In [ ]:
df.select(
    pl.col("a").str2.similar_words(
        vocab = pl.col("b"),
        k = 2,
        metric = "lv"
    ).alias("similar_words_from_vocab"),
)

In [ ]:
df.select(
    pl.col("a").str2.similar_words(
        vocab = pl.col("b"),
        k = 2,
        threshold = 4, # <= threshold hamming distance away
        metric = "hamming"
    ).alias("similar_words_from_vocab"),
)

In [ ]:
df.select(
    pl.col("a").str2.similar_words(
        vocab = ["WORLD", "AAAAA", "ABCDEFG", "ZIV"],
        k = 2,
        metric = "lv"
    ).alias("similar_words_from_vocab"),
)

In [ ]:
df = pl.DataFrame({
    "a": range(100_000),
    "b": range(100_000),
    "c": range(100_000),
})

In [ ]:
df = pl.DataFrame({
    "y": [[1,0,0],[0,1,0],[0,0,1],[1,0,0],[0,1,0]],
    "pred":[[0.1, 0.5, 0.4], [0.2, 0.6, 0.2], [0.4, 0.1, 0.5], [0.9, 0.05, 0.05], [0.2, 0.5, 0.3]]
})

In [ ]:
df.select(
    pl.col("pred").list.get(pl.col("y"))
)

In [ ]:
df.select(
    pl.col("y").metric.categorical_cross_entropy(pl.col("pred"), normalize=True, dense = False)
).item(0,0)